<a href="https://colab.research.google.com/github/valentinocc/Keras_cifar10/blob/master/keras_rnn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import keras
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Activation, add, MaxPooling2D, Dense, Flatten, Input, BatchNormalization, AveragePooling2D
from keras.models import Model

%load_ext tensorboard.notebook

Using TensorFlow backend.


In [0]:
subtract_pixel_mean = True

In [0]:
def residual_block(x, filter_amount, downsample=False):  

  if (downsample):#project x_input to the new dimensions
    x_input = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
    x_input = Conv2D(filter_amount, (1, 1), strides = (1, 1))(x_input)
    x = conv_elu_block(x, filter_amount, strides = (2, 2))
    
  else:
    x_input = x
    x = conv_elu_block(x, filter_amount)
  
  x = conv_elu_block(x, filter_amount)
  
  residual_block = add([x, x_input])
  
  x = Activation('relu')(x)
  
  return residual_block
  
  
def conv_elu_block(x, filter_amount, strides = (1,1)):
  
  x = Conv2D(filter_amount, (3, 3), strides = strides, padding = 'same')(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
  
  return x


def residual_neural_network(input_shape, num_res_blocks, class_amount):
  
  inputs = Input(shape=input_shape)
  filter_amount = 16
    
  x = Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_initializer='glorot_uniform')(inputs)
  
  for stack in range(3):
    for res_block in range(num_res_blocks):
      if (stack != 0 and res_block == 0):
        x = residual_block(x, filter_amount, downsample = True)
      else:
        x = residual_block(x, filter_amount, downsample = False)
        
    filter_amount *= 2
    
  x = AveragePooling2D(pool_size = 8)(x)
  y = Flatten()(x)
  outputs = Dense(class_amount, activation = 'softmax')(y)
  
  model = Model(inputs=inputs, outputs=outputs)
  
  return model

def learning_rate_schedule(epoch):
  
  lr = 1e-3
  if (epoch > 180):
    lr *= 5e-4
  elif (epoch > 160):
    lr *= 1e-3
  elif (epoch > 120):
    lr *= 1e-2
  elif (epoch > 80):
    lr *= 1e-1
  
  print('learning rate: ', lr)
  
  return lr



In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.006*8, shuffle= True)

# Normalize data.
x_train = x_train.astype('float32') / 255
x_valid = x_valid.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_valid -= x_train_mean
    x_test -= x_train_mean


170500096/170498071 [==============================] - 9s 0us/step


In [5]:
INPUT_SHAPE = x_train.shape[1:]
STACK_AMOUNT = 9
CLASS_AMOUNT = 10

model = residual_neural_network(INPUT_SHAPE, STACK_AMOUNT, CLASS_AMOUNT)


W0618 20:07:00.977742 140260532844416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0618 20:07:01.014117 140260532844416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0618 20:07:01.021682 140260532844416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0618 20:07:01.073374 140260532844416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0618 20:07:01.074193 1402605328

In [0]:
EPOCHS = 5
BATCH_SIZE = 32


model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])


EPOCHS = 200
datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=5,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.2,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(x_test, y_test), 
                    steps_per_epoch = x_train.shape[0] / BATCH_SIZE,
                    epochs=EPOCHS, verbose=1)
    
model.evaluate(x_test, y_test, batch_size = BATCH_SIZE)

W0618 20:07:08.700874 140260532844416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0618 20:07:09.246635 140260532844416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
1488/1487 [==============================] - 125s 84ms/step - loss: 1.5256 - acc: 0.4619 - val_loss: 1.1694 - val_acc: 0.5918
Epoch 2/200
1488/1487 [==============================] - 110s 74ms/step - loss: 0.9937 - acc: 0.6524 - val_loss: 0.9540 - val_acc: 0.6913
Epoch 3/200
1488/1487 [==============================] - 110s 74ms/step - loss: 0.8088 - acc: 0.7177 - val_loss: 0.7652 - val_acc: 0.7403
Epoch 4/200
 651/1487 [============>.................] - ETA: 58s - loss: 0.7238 - acc: 0.7493